In [2]:
import Pkg
Pkg.add("Cbc")
Pkg.add("JuMP")

   Updating registry at `C:\Users\aebou\.julia\registries\General`
  Resolving package versions...
  Installed CoinUtils_jll ── v2.11.3+4
  Installed OpenBLAS32_jll ─ v0.3.9+4
  Installed Cgl_jll ──────── v0.60.2+6
  Installed Osi_jll ──────── v0.108.5+4
  Installed Clp_jll ──────── v1.17.6+7
  Installed Cbc_jll ──────── v2.10.5+2
  Installed Cbc ──────────── v0.8.0
Updating `C:\Users\aebou\.julia\environments\v1.5\Project.toml`
  [9961bab8] + Cbc v0.8.0
Updating `C:\Users\aebou\.julia\environments\v1.5\Manifest.toml`
  [9961bab8] + Cbc v0.8.0
  [38041ee0] + Cbc_jll v2.10.5+2
  [3830e938] + Cgl_jll v0.60.2+6
  [06985876] + Clp_jll v1.17.6+7
  [be027038] + CoinUtils_jll v2.11.3+4
  [656ef2d0] + OpenBLAS32_jll v0.3.9+4
  [7da25872] + Osi_jll v0.108.5+4
   Building Cbc → `C:\Users\aebou\.julia\packages\Cbc\JdG60\deps\build.log`


In [1]:
using JuMP
using Cbc
using DelimitedFiles
using NBInclude

# constante epsilon
const ϵ = 0.00001

1.0e-5

# Projet Programmation Mathématiques Avancée et Applications
Hugo Miccinilli, Aboa Bouadou, Florian Jacta, Theo DI PIAZZA | 4GM

## Kidney exchange problem

### Introduction et formulation du problème

On s'intéresse au problème de don croisé de rein.  
**Qu'est ce que le don croisé ?**   
"Il arrive assez souvent qu’une personne soit volontaire pour faire don d’un rein à un proche, parent, conjoint ou ami, mais que le don soit impossible en raison d’une incompatibilité tissulaire avec le receveur. On appellera ces deux personnes la paire A. Une situation identique peut se retrouver ailleurs au sein d’une paire B. Il arrive parfois que le donneur de la paire A soit compatible avec le receveur de la paire B, et vice-versa que le donneur de la paire B soit compatible avec le receveur B. Cette situation – peu courante – peut permettre un « don croisé »." (source France-adot)  

**Représentation mathématique du problème**  
Lorsqu'il y a plusieurs paires patient-donneur, l'opération de greffe est plus compliquée à mettre en place. Mathématiquement, on considère un graphe orienté simple $G = (V,A)$ où V représente l'ensemble des couples patient-donneur $(P_i,D_i)$ et A représente les compatibilités entre entre les différentes paires, c'est à dire $(i,j) \in A$ si le donneur $D_i$ est compatible avec le patient $P_j$. Certains patients sont prioritaires par rapport à d'autres donc on affecte un score $w_{i,j}$ qui caractérise ce fait. Plus le score est elevé, plus le patient impliqué dans l'arc est prioritaire. Ce score peut être vu comme l'utilité d'un arc donné.

**Formulation**  
Comme indiqué dans l'énoncé, on peut associer des patients et des donneurs de sorte à former un cycle dans G. Pour des raisons opérationnelles (les différents greffes dans un cycle doivent se dérouler en même temps), il ne faut pas qu'un cycle soit trop long. Soit L la longueur maximale qu'on peut avoir dans un cycle choisi et soit $\mathcal{C}_L$ l'ensemble des cycles de G tels que $|c| \le L$ pour $c \in \mathcal{C}_L$. On définit $\mathcal{C}_L(i)$ comme l'ensemble des cycles qui contiennent un sommet $i \in V$ et $w_c = \sum \limits_{(i,j) \in c} w_{i,j}$ pour $c \in \mathcal{C}_L$. L'objectif du problème est de choisir l'ensemble des cycles qui permettra de maximiser l'utilité. Certains patients peuvent donc ne pas avoir de donneurs après résolution du problème. 

Le problème peut ainsi se formuler comme suit :  

*Variables*  
$\forall c \in \mathcal{C}_L, z_c = \{0,1\} = 1$ si le cycle c est choisi, 0 sinon  

*Objectif*  
$\sum \limits_{c \in \mathcal{C}_L} w_cz_c$ 

*Contraintes*  
$\sum \limits_{c \in \mathcal{C}_L(i)} z_c \le 1, \forall i \in V$  
$z \in \{0,1\}^{|\mathcal{C}_L|}$


Le problème ayant un très grand nombre de variables, on veut le résoudre en utilisant l'algorithme de branch and price. 

### Initialisation du problème

Avant de coder l'algorithme, on code quelques classes et fonctions qui seront utiles pour initialiser les instances du problème. On explique en détail ces fonctions dans les différents fichiers qu'on a inclut ci-dessous.

In [2]:
@nbinclude("dataparser.ipynb")
@nbinclude("classes.ipynb")
@nbinclude("traitement_noeuds.ipynb")

   Updating registry at `C:\Users\aebou\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aebou\.julia\environments\v1.5\Manifest.toml`

There is an arc going from node 1 to 3. It is: Edge 1 => 3
There is an arc going from node 1 to 16. It is: Edge 1 => 16
There is an arc going from node 2 to 4. It is: Edge 2 => 4
There is an arc going from node 2 to 14. It is: Edge 2 => 14
There is an arc going from node 3 to 4. It is: Edge 3 => 4
There is an arc going from node 3 to 14. It is: Edge 3 => 14
There is an arc going from node 4 to 6. It is: Edge 4 => 6
There is an arc going from node 4 to 13. It is: Edge 4 => 13
There is an arc going from node 4 to 16. It is: Edge 4 => 16
There is an arc going from node 5 to 1. It is: Edge 5 => 1
There is an arc going from node 5 to 2. It is: Edge 5 => 2
There is an arc going from node 5 to 4. It is: Edge 5 => 4
There is an arc going from node 5 to 6. It is: Edge 5 => 6
There is an arc going from node 5 to 8. It is: Edge 5 => 8
There is an arc going from node 5 to 10. It is: Edge 5 => 10
There is an arc going from node 5 to 11. It is: Edge 5 => 11
There is an arc going from node 5 to 13. I

master (generic function with 1 method)

In [5]:
# Pour chaque paire, on va récupérer les arcs dont ils sont l'origine (calcul du demi-degré extérieur des sommets du graphe)
function degreExt(kep)
    origine = Array{Array{Int64,1},1}()
    for v in 1:nv(kep)
        # On recupère les indices des arcs dont l'origine est v
        origin_v = Array{Int64,1}()
        for (ind,arc) in enumerate(edges(kep))
            if src(arc) == v
                push!(origin_v,ind)
            end
        end
        push!(origine,origin_v)
    end
    return origine
end

# Pour chaque paire, on va récupérer les arcs dont ils sont la destination (calcul du demi-degré intérieur des sommets)
function degreInt(kep)
    dest = Array{Array{Int64,1},1}()
    for v in 1:nv(kep)
        # On recupère les indices des arcs dont l'origine est v
        dest_v = Array{Int64,1}()
        for (ind,arc) in enumerate(edges(kep))
            if dst(arc) == v
                push!(dest_v,ind)
            end
        end
        push!(dest,dest_v)
    end
    return dest
end


degreInt (generic function with 1 method)

In [13]:
for edge = collect(edges(pool))
    print(edge,"\n")
end

Edge 1 => 3
Edge 1 => 16
Edge 2 => 4
Edge 2 => 14
Edge 3 => 4
Edge 3 => 14
Edge 4 => 6
Edge 4 => 13
Edge 4 => 16
Edge 5 => 1
Edge 5 => 2
Edge 5 => 4
Edge 5 => 6
Edge 5 => 8
Edge 5 => 10
Edge 5 => 11
Edge 5 => 13
Edge 5 => 14
Edge 5 => 16
Edge 6 => 4
Edge 6 => 14
Edge 7 => 2
Edge 7 => 4
Edge 7 => 5
Edge 7 => 8
Edge 7 => 10
Edge 7 => 11
Edge 7 => 14
Edge 7 => 16
Edge 8 => 4
Edge 8 => 14
Edge 9 => 6
Edge 9 => 15
Edge 10 => 4
Edge 10 => 14
Edge 11 => 4
Edge 11 => 14
Edge 13 => 4
Edge 13 => 14
Edge 14 => 3
Edge 14 => 6
Edge 14 => 13
Edge 14 => 15
Edge 14 => 16
Edge 15 => 4
Edge 15 => 14
Edge 16 => 4


In [14]:
degreInt(pool)

16-element Array{Array{Int64,1},1}:
 [10]
 [11, 22]
 [1, 40]
 [3, 5, 12, 20, 23, 30, 34, 36, 38, 45, 47]
 [24]
 [7, 13, 32, 41]
 []
 [14, 25]
 []
 [15, 26]
 [16, 27]
 []
 [8, 17, 42]
 [4, 6, 18, 21, 28, 31, 35, 37, 39, 46]
 [33, 43]
 [2, 9, 19, 29, 44]

In [ ]:
degreExt(pool)

Le sous problème associé au master s'écrit :
$$min \sum \limits_{a \in A} w_ax_a - \sum \limits_{i \in V} \pi_iy_i \\ s.l.c \sum \limits_{ a \in \delta^+(i)} x_a =   \sum \limits_{ a \in \delta^-(i)} x_a \text{,  } \forall i \in V \\ \sum \limits_{a \in A} x_a \le L \\ \sum \limits_{ a \in \delta^-(i)} x_a = y_i \text{,  } \forall i \in V \\ x \in \{0,1\}^{|A|} \\ y \in \{0,1\}^{|V|}$$

On cherche donc un cycle de longueur maximale égale à L qui minimise le coût réduit.
La fonction permettant d'initialiser le sous-problème est écrite dans le fichier *traitement_noeud*